In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Data Processing

In [18]:
TRAIN_PATH = "../input/tensorflow-great-barrier-reef/train_images"
HEIGHT, WIDTH = 720, 1280
image_size = 640

vid0_path = "../input/tensorflow-great-barrier-reef/train_images/video_0"
vid1_path = "../input/tensorflow-great-barrier-reef/train_images/video_1"
vid2_path = "../input/tensorflow-great-barrier-reef/train_images/video_2"
vid_paths = [vid0_path, vid1_path, vid2_path]

vid0_ls = [os.path.join(vid0_path,f) for f in os.listdir(vid0_path)]
vid0_ls = sorted(vid0_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid1_ls = [os.path.join(vid1_path,f) for f in os.listdir(vid1_path)]
vid1_ls = sorted(vid1_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))

vid2_ls = [os.path.join(vid2_path,f) for f in os.listdir(vid2_path)]
vid2_ls = sorted(vid2_ls, key=lambda x: int("".join([i for i in x if i.isdigit()])))
files_ls = [vid0_ls, vid1_ls, vid2_ls]

train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv",
                       sep = r',', skipinitialspace = True)

In [19]:
print(vid0_ls[:3])
print("\n",files_ls[0][:3])

In [20]:
len(files_ls[0])

In [21]:
train_df.head(5)

#### Helper functions

In [22]:
def get_oldpath(x):
    if x.video_id == 0:
        path = os.path.join(vid0_path,str(x.video_frame)+".jpg")
    elif x.video_id == 1:
        path = os.path.join(vid1_path,str(x.video_frame)+".jpg")
    else:
        path = os.path.join(vid2_path,str(x.video_frame)+".jpg")
        
    return path

def get_newpath(x):
    filename = f"{x.video_id}_{x.video_frame}.jpg"
    return os.path.join("./dataset", filename)

def get_filename(x):
    return f"{x.video_id}_{x.video_frame}.jpg"

In [23]:
import ast
from tqdm import tqdm
import shutil
train_df = train_df[train_df.annotations != "[]"]
train_df["annotations"] = train_df["annotations"].map(lambda x : ast.literal_eval(x))

train_df["filepath"] = train_df.apply(lambda x : get_oldpath(x), axis=1)
train_df["newpath"] = train_df.apply(lambda x : get_newpath(x), axis=1)
train_df["filename"] = train_df.apply(lambda x : get_filename(x), axis=1)

os.makedirs("./dataset",exist_ok=True)

for i in tqdm(range(len(train_df))):
    src = train_df.iloc[i]["filepath"]
    dst = train_df.iloc[i]["newpath"]
    shutil.copy(src,dst)
    
train_df.head(3)

#### Main Dataframe

In [35]:
df = train_df


df = df.explode("annotations")

df["width"] = [WIDTH]*len(df)
df["height"] = [HEIGHT]*len(df)
df["label"] = ["starfish"]*len(df)

df["xmin"] = df.apply(lambda x : x.annotations["x"], axis=1)
df["ymin"] = df.apply(lambda x : x.annotations["y"], axis=1)
df["xmax"] = df.apply(lambda x : x.annotations["x"]+x.annotations["width"], axis=1)
df["ymax"] = df.apply(lambda x : x.annotations["y"]+x.annotations["height"], axis=1)


df.loc[df["xmax"] > 1280, "xmax"] = 1280
df.loc[df["ymax"] > 720, "ymax"] = 720

df = df.drop(["video_id","sequence","video_frame","sequence_frame","image_id","annotations","filepath","newpath"], axis=1)
df = df.reset_index(drop=True)

print(df.head(5))

In [25]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import cv2

In [26]:
def draw_annotations(spot_fish_img, img_row):
    spot_fish_img = cv2.rectangle(spot_fish_img, (img_row['xmin'], img_row['ymin']),
              (img_row['xmax'], img_row['ymax']), (255,255,0), 2)
    return spot_fish_img

In [88]:
img_row = df.iloc[random.randint(0, len(df))]
img_row

In [89]:
original_img_path = os.path.join("./dataset", img_row['filename'])
original_img = npimg.imread(original_img_path)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
fig.tight_layout()

axs.imshow(original_img)
axs.set_title('Original Image')

In [91]:
spot_fish_img = original_img.copy()
draw_annotations(spot_fish_img, img_row)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
fig.tight_layout()

axs.imshow(spot_fish_img)
axs.set_title('spotfish Image')

# Data Modelling

Ref: https://www.kaggle.com/amyrouillard/object-detection-great-barrier-reef/edit

Load data: cots-detection-w-tensorflow-object-detection-api

In [104]:
import sys
import tensorflow as tf

MODEL_DIR = '../input/cots-detection-w-tensorflow-object-detection-api/cots_efficientdet_d0'

tf.keras.backend.clear_session()
detect_fn_tf_odt = tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'saved_model'))

#IMG_SIZE = (720, 1280)
#IMG_SHAPE = IMG_SIZE + (3,)
#detect_fn_tf_odt = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                               include_top=False,
#                                               weights='imagenet')



In [107]:
from PIL import Image

DETECTION_THRESHOLD = 0.3

test_img_row = df.iloc[random.randint(0, len(df))]
test_img_path = os.path.join("./dataset", test_img_row['filename'])

image_np = np.array(Image.open(test_img_path))
#print(image_np.shape)

input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn_tf_odt(input_tensor)

num_detections = detections['num_detections'][0].numpy().astype(np.int32)
predictions = []
for index in range(num_detections):
    score = detections['detection_scores'][0][index].numpy()
    if score < DETECTION_THRESHOLD:
            continue
    bbox = detections['detection_boxes'][0][index].numpy()
    y_min = int(bbox[0] * height)
    x_min = int(bbox[1] * width)
    y_max = int(bbox[2] * height)
    x_max = int(bbox[3] * width)

    bbox_width = x_max - x_min
    bbox_height = y_max - y_min

    #predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    predictions.append([score, x_min, y_min, bbox_width, bbox_height])
#print('Prediction:', ', '.join(predictions))
#print('Prediction:', predictions)

test_img = npimg.imread(test_img_path)
spot_fish_img = test_img.copy()
draw_annotations(spot_fish_img, img_row)

for x in predictions:
    spot_fish_img = cv2.rectangle(spot_fish_img, (x[1], x[2]), (x[1]+x[3], x[2]+x[4]),(255, 0, 0), 2)

fig, axs = plt.subplots(1, 1, figsize=(15, 10))
#fig.tight_layout()

axs.imshow(spot_fish_img)
axs.set_title('spotfish Image')



# Evaluation

# Submission 